# 14 - ANOVA

<center><img src="img/anovaa.png" alt="Testy różnic" width="500"/></center>

# ANOVA w Pythonie – wprowadzenie

Analiza wariancji (**ANOVA**, *Analysis of Variance*) służy do sprawdzania, czy **średnie kilku niezależnych grup** różnią się od siebie istotnie statystycznie. Porównujemy dwa źródła zmienności:

- **Między grupami (Between Groups)** – zmienność wynikająca z różnic między średnimi grup  
- **Wewnątrz grup (Within Groups / Residual)** – zmienność wynikająca z indywidualnych różnic w obrębie każdej grupy  

Statystyka testowa ma postać:

$$
F = \frac{MS_{\text{between}}}{MS_{\text{within}}}
$$

gdzie:

$$
MS = \frac{SS}{df}
$$

Duża wartość statystyki *F* sugeruje, że średnie w grupach różnią się bardziej, niż wynikałoby to z przypadku.

---

## Hipotezy w jednoczynnikowej ANOVA

- **H₀:** wszystkie średnie populacyjne są równe  
$$
  \mu_1 = \mu_2 = \dots = \mu_k
$$

- **H₁:** co najmniej dwie średnie różnią się między sobą

---

## Tabela ANOVA 

| Źródło zmienności | SS (Sum of Squares) | df  | MS (Mean Square)              | F                           | p-value |
|-------------------|---------------------|-----|-------------------------------|-----------------------------|---------|
| Between Groups    | SS_between          | k-1 | SS_between/(k-1)              | MS_between/MS_within        | p       |
| Within Groups     | SS_within           | N-k | SS_within/(N-k)               | —                           | —       |
| Total             | SS_total            | N-1 | —                             | —                           | —       |

Gdzie:  
- `k` – liczba porównywanych grup  
- `N` – łączna liczba obserwacji

---

<center><img src="img/anova.png" alt="Testy różnic" width="500"/></center>